# Import Required Libraries
Import the necessary libraries, including yfinance, pandas, numpy, and datetime.

In [24]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from statsmodels.tsa.stattools import adfuller
from scipy.stats import jarque_bera

# Define Tickers and Date Range
Define the tickers for USD/INR and Sensex, as well as the date range for the data.

In [25]:
# Define the tickers for USD/INR and Sensex
usd_inr_ticker = 'INR=X'
sensex_ticker = '^BSESN'

# Define the date range
start_date = '2004-01-01'
end_date = '2015-12-31'

# Fetch Data
Fetch the data for USD/INR and Sensex using yfinance.

In [26]:
# Fetch Data

# Fetch the data for USD/INR and Sensex using yfinance
usd_inr_data = yf.download(usd_inr_ticker, start=start_date, end=end_date)
sensex_data = yf.download(sensex_ticker, start=start_date, end=end_date)

# Display the first few rows of the data
print(usd_inr_data.tail())
print(sensex_data.tail())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price                      Adj Close      Close       High        Low  \
Ticker                         INR=X      INR=X      INR=X      INR=X   
Date                                                                    
2015-12-24 00:00:00+00:00  66.153999  66.153999  66.094002  65.660004   
2015-12-25 00:00:00+00:00  65.940002  65.940002  66.070000  65.940002   
2015-12-28 00:00:00+00:00  65.940002  65.940002  66.208000  65.940002   
2015-12-29 00:00:00+00:00  66.134003  66.134003  66.408997  66.099998   
2015-12-30 00:00:00+00:00  66.260002  66.260002  66.464996  66.139999   

Price                           Open Volume  
Ticker                         INR=X  INR=X  
Date                                         
2015-12-24 00:00:00+00:00  66.094002      0  
2015-12-25 00:00:00+00:00  65.940002      0  
2015-12-28 00:00:00+00:00  65.959999      0  
2015-12-29 00:00:00+00:00  66.141998      0  
2015-12-30 00:00:00+00:00  66.265999      0  
Price                         Adj Close        

# Match Dates
Align the dates for USD/INR and Sensex data to ensure they match.

In [27]:
# Align the dates for USD/INR and Sensex data to ensure they match

# Handle missing values by forward filling and then backward filling
# usd_inr_data = usd_inr_data.ffill().bfill()
# sensex_data = sensex_data.ffill().bfill()

# Ensure both datasets have the same date range by taking the intersection of their indices
common_dates = usd_inr_data.index.intersection(sensex_data.index)
usd_inr_data = usd_inr_data.loc[common_dates]
sensex_data = sensex_data.loc[common_dates]

# Display the first few rows of the cleaned data
print(usd_inr_data.head())
print(sensex_data.head())

Price                      Adj Close      Close       High        Low  \
Ticker                         INR=X      INR=X      INR=X      INR=X   
Date                                                                    
2004-01-02 00:00:00+00:00  45.603001  45.603001  45.645000  45.497002   
2004-01-05 00:00:00+00:00  45.466000  45.466000  45.564999  45.459000   
2004-01-06 00:00:00+00:00  45.549000  45.549000  45.604000  45.457001   
2004-01-07 00:00:00+00:00  45.387001  45.387001  45.560001  45.375000   
2004-01-08 00:00:00+00:00  45.467999  45.467999  45.490002  45.345001   

Price                           Open Volume  
Ticker                         INR=X  INR=X  
Date                                         
2004-01-02 00:00:00+00:00  45.527000      0  
2004-01-05 00:00:00+00:00  45.528000      0  
2004-01-06 00:00:00+00:00  45.467999      0  
2004-01-07 00:00:00+00:00  45.548000      0  
2004-01-08 00:00:00+00:00  45.391998      0  
Price                        Adj Close        C

# Take Natural Logarithm
Compute the natural logarithm of the cleaned and aligned data.

In [28]:
# Calculate log returns
usd_inr_returns = np.log(usd_inr_data['Close'] / usd_inr_data['Close'].shift(1)).dropna()
sensex_returns = np.log(sensex_data['Close'] / sensex_data['Close'].shift(1)).dropna()

# Combine the returns into a single DataFrame
returns = pd.concat([usd_inr_returns, sensex_returns], axis=1)
returns.columns = ['USD_INR_Returns', 'Sensex_Returns']
returns = returns.dropna()

# Display the first few rows of the returns
print(returns.head())

                           USD_INR_Returns  Sensex_Returns
Date                                                      
2004-01-05 00:00:00+00:00        -0.003009        0.002057
2004-01-06 00:00:00+00:00         0.001824       -0.015917
2004-01-07 00:00:00+00:00        -0.003563        0.002249
2004-01-08 00:00:00+00:00         0.001783        0.025117
2004-01-09 00:00:00+00:00        -0.002576        0.001807


In [29]:
# Calculate descriptive statistics
descriptive_stats = returns.describe()

# Calculate skewness and kurtosis
skewness = returns.skew()
kurtosis = returns.kurtosis()

# Perform Jarque-Bera test
jb_test_usd_inr = jarque_bera(returns['USD_INR_Returns'])
jb_test_sensex = jarque_bera(returns['Sensex_Returns'])

# Display the descriptive statistics
print("Descriptive Statistics:")
print(f"Mean:\n{returns.mean()}")
print(f"\nMaximum:\n{returns.max()}")
print(f"\nMinimum:\n{returns.min()}")
print(f"\nStd. Dev.:\n{returns.std()}")
print(f"\nSkewness:\n{skewness}")
print(f"\nKurtosis:\n{kurtosis}")

# Display the Jarque-Bera test results
print("\nJarque-Bera Test:")
print(f"USD/INR Returns: Statistic = {jb_test_usd_inr[0]}, p-value = {jb_test_usd_inr[1]}")
print(f"Sensex Returns: Statistic = {jb_test_sensex[0]}, p-value = {jb_test_sensex[1]}")

Descriptive Statistics:
Mean:
USD_INR_Returns    0.000128
Sensex_Returns     0.000500
dtype: float64

Maximum:
USD_INR_Returns    0.060972
Sensex_Returns     0.159900
dtype: float64

Minimum:
USD_INR_Returns   -0.060972
Sensex_Returns    -0.118092
dtype: float64

Std. Dev.:
USD_INR_Returns    0.005534
Sensex_Returns     0.015417
dtype: float64

Skewness:
USD_INR_Returns   -0.080399
Sensex_Returns    -0.089932
dtype: float64

Kurtosis:
USD_INR_Returns    20.222877
Sensex_Returns      8.835082
dtype: float64

Jarque-Bera Test:
USD/INR Returns: Statistic = 49631.379407971166, p-value = 0.0
Sensex Returns: Statistic = 9473.943444564695, p-value = 0.0


In [30]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test for USD/INR returns
adf_usd_inr = adfuller(returns['USD_INR_Returns'])
print('Augmented Dickey-Fuller Test: USD/INR Returns')
print(f'ADF Test Statistic: {adf_usd_inr[0]:.6e}')
print(f'p-value: {adf_usd_inr[1]:.6e}')
print(f'# Lags Used: {adf_usd_inr[2]:.6e}')
print(f'# Observations Used: {adf_usd_inr[3]:.6e}')
for key, value in adf_usd_inr[4].items():
    print(f'Critical Value ({key}): {value:.6e}')
print()

# Perform ADF test for Sensex returns
adf_sensex = adfuller(returns['Sensex_Returns'])
print('Augmented Dickey-Fuller Test: Sensex Returns')
print(f'ADF Test Statistic: {adf_sensex[0]:.6e}')
print(f'p-value: {adf_sensex[1]:.6e}')
print(f'# Lags Used: {adf_sensex[2]:.6e}')
print(f'# Observations Used: {adf_sensex[3]:.6e}')
for key, value in adf_sensex[4].items():
    print(f'Critical Value ({key}): {value:.6e}')

Augmented Dickey-Fuller Test: USD/INR Returns
ADF Test Statistic: -2.263422e+01
p-value: 0.000000e+00
# Lags Used: 4.000000e+00
# Observations Used: 2.918000e+03
Critical Value (1%): -3.432593e+00
Critical Value (5%): -2.862531e+00
Critical Value (10%): -2.567298e+00

Augmented Dickey-Fuller Test: Sensex Returns
ADF Test Statistic: -3.937220e+01
p-value: 0.000000e+00
# Lags Used: 1.000000e+00
# Observations Used: 2.921000e+03
Critical Value (1%): -3.432591e+00
Critical Value (5%): -2.862530e+00
Critical Value (10%): -2.567297e+00


In [34]:
# Correlation test
correlation_matrix = returns.corr()
print(correlation_matrix)

                 USD_INR_Returns  Sensex_Returns
USD_INR_Returns         1.000000       -0.235629
Sensex_Returns         -0.235629        1.000000


In [32]:
from statsmodels.tsa.stattools import coint

# Perform co-integration test
coint_test = coint(returns['USD_INR_Returns'], returns['Sensex_Returns'])
print('Co-integration test statistic:', coint_test[0])
print('p-value:', coint_test[1])

Co-integration test statistic: -23.313561004598863
p-value: 0.0


In [33]:
from statsmodels.tsa.api import VAR

# Fit the VAR model
model = VAR(returns)
results = model.fit(maxlags=15, ic='aic')

# Display the summary of the model
print(results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 07, Nov, 2024
Time:                     22:31:26
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -18.8441
Nobs:                     2918.00    HQIC:                  -18.8729
Log likelihood:           19300.3    FPE:                6.25971e-09
AIC:                     -18.8891    Det(Omega_mle):     6.21278e-09
--------------------------------------------------------------------
Results for equation USD_INR_Returns
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                      0.000196         0.000099            1.979           0.048
L1.USD_INR_Returns        -0.207641         0.019059          -10.894           0.000
L1.Sensex_Returns         -0.0

e:\Rouhin Projects\VaRmodel\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
